<a href="https://colab.research.google.com/github/SunSlick2/MoveEmails/blob/main/MoveArchiveEmailsFromRootToInbox2_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import win32com.client
import pythoncom
import logging
import time
from datetime import datetime
import json
import os
import tkinter as tk
from tkinter import messagebox

# =========================================================================
# THROTTLING CONFIGURATION
# Adjust these values based on your server's observed limits.
# The script will reset the COM session after SESSION_RESET_THRESHOLD moves.
# =========================================================================
MOVE_BATCH_SIZE = 120           # Number of items to move before a short internal pause.
MOVE_PAUSE_SECONDS = 10         # Short pause after a batch move (internal throttle).
SESSION_RESET_THRESHOLD = 240   # Total moves allowed before forcing a full COM session reset.
RECONNECT_DELAY_SECONDS = 60    # Long pause after a session reset (long cool-down).
# =========================================================================

class EmailMigrator:
    def __init__(self):
        self.setup_logging()
        self.migration_report = {
            'start_time': None,
            'end_time': None,
            'source_store': 'Online Archive - ghi.jkl@def.com',
            'destination_folder': 'Inbox',
            'total_attempted': 0,
            'total_successful': 0,
            'total_failed': 0,
            'failed_items': [],
            'config': {
                'batch_size': MOVE_BATCH_SIZE,
                'pause_seconds': MOVE_PAUSE_SECONDS,
                'reset_threshold': SESSION_RESET_THRESHOLD,
                'reconnect_delay': RECONNECT_DELAY_SECONDS
            }
        }

    def setup_logging(self):
        """Setup comprehensive logging"""
        log_dir = "migration_logs"
        os.makedirs(log_dir, exist_ok=True)
        log_filename = os.path.join(log_dir, f'archive_migrator_{datetime.now().strftime("%Y%m%d_%H%M%S")}.log')

        logging.basicConfig(
            filename=log_filename,
            level=logging.INFO,
            format='%(asctime)s - %(levelname)s - %(message)s'
        )
        # Also log to console
        console = logging.StreamHandler()
        console.setLevel(logging.INFO)
        formatter = logging.Formatter('%(levelname)s: %(message)s')
        console.setFormatter(formatter)
        logging.getLogger().addHandler(console)
        logging.info(f"Logging to: {log_filename}")

    def find_store_by_display_name(self, namespace, display_name):
        """Finds an Outlook store (mailbox/archive) by its display name."""
        try:
            for store in namespace.Stores:
                if store.DisplayName == display_name:
                    return store
        except Exception as e:
            logging.error(f"Error iterating stores: {e}", exc_info=True)
        return None

    def get_root_item_list(self, source_folder):
        """Retrieves a list of EntryIDs for items in the root folder, filtering for MailItems."""
        item_list = []
        try:
            items = source_folder.Items
            for item in items:
                # olMail (Class 43) is the primary item type we want to move
                if getattr(item, 'Class', None) == 43:
                    item_list.append(item.EntryID)
        except Exception as e:
            logging.error(f"Error retrieving items from root folder: {e}", exc_info=True)
        return item_list

    def migrate_items(self, outlook, namespace, items_to_move_ids):
        """
        Moves a subset of items, respecting the internal throttling configuration.
        Returns the number of items successfully moved in this call.
        """

        # Re-locate the folders within the current session
        archive_store = self.find_store_by_display_name(namespace, self.migration_report['source_store'])
        if not archive_store:
            raise ValueError("Archive store not found during migration step.")

        source_folder = archive_store.GetRootFolder()
        try:
            # Destination is hardcoded to Inbox within the Online Archive
            destination_folder = source_folder.Folders['Inbox']
        except Exception:
            # Handle case where 'Inbox' doesn't exist (shouldn't happen in a standard archive)
            raise ValueError("Inbox folder not found in the Online Archive root.")

        logging.info(f"Targeting source folder: {source_folder.Name} and destination: {destination_folder.Name}")

        items = source_folder.Items
        moved_count_session = 0

        # Determine the batch size for this specific move run
        # We cap the items to move at the SESSION_RESET_THRESHOLD or less
        # based on how many are remaining in the list.
        max_items_to_process = min(len(items_to_move_ids), SESSION_RESET_THRESHOLD)

        logging.info(f"Attempting to move up to {max_items_to_process} items in this session.")

        for i in range(max_items_to_process):
            entry_id = items_to_move_ids[i]

            try:
                # Get item by EntryID
                item = namespace.GetItemFromID(entry_id)

                self.migration_report['total_attempted'] += 1

                subject = getattr(item, 'Subject', 'N/A')

                # Perform the move
                item.Move(destination_folder)

                moved_count_session += 1
                self.migration_report['total_successful'] += 1

                logging.info(f"SUCCESS ({self.migration_report['total_successful']}): {subject[:70]}...")

                # Apply internal batch throttling
                if moved_count_session > 0 and moved_count_session % MOVE_BATCH_SIZE == 0:
                    logging.info(f"Hit internal batch size ({MOVE_BATCH_SIZE}). Pausing for {MOVE_PAUSE_SECONDS} seconds...")
                    time.sleep(MOVE_PAUSE_SECONDS)

            except Exception as e:
                # This catches the COM error (-2147352567) and other immediate move errors
                self.migration_report['total_failed'] += 1

                # Log the specific error details
                error_details = str(e)
                if "-2147352567" in error_details:
                    error_type = "Server Throttling/COM Error"
                else:
                    error_type = "General Move Error"

                self.migration_report['failed_items'].append({
                    'subject': subject,
                    'error': error_details,
                    'type': error_type
                })

                logging.error(f"FAILURE ({self.migration_report['total_failed']}): {subject[:70]}... Error: {error_details}")

                # Return the number of items successfully moved *in this run* and stop,
                # forcing the outer loop to trigger the connection reset.
                return moved_count_session

        return moved_count_session

    def run_migration(self):
        """
        Main migration function with a persistent loop that resets the COM session
        to counter cumulative server throttling.
        """
        logging.info("Starting Online Archive Root to Inbox migration with aggressive throttling.")
        self.migration_report['start_time'] = datetime.now().isoformat()

        items_to_move_ids = []
        is_initialized = False

        while True:
            outlook = None
            try:
                # 1. Initialize COM and Outlook objects
                if not is_initialized:
                    pythoncom.CoInitialize()
                    is_initialized = True

                outlook = win32com.client.Dispatch("Outlook.Application")
                namespace = outlook.GetNamespace("MAPI")

                # 2. Locate the Archive Store and Folders
                archive_store = self.find_store_by_display_name(namespace, self.migration_report['source_store'])
                if not archive_store:
                    messagebox.showerror("Initialization Error", f"Online Archive '{self.migration_report['source_store']}' not found. Ensure it is connected.")
                    raise ValueError("Archive store not found.")

                logging.info(f"Store found: '{archive_store.DisplayName}'")

                # 3. Get the list of remaining items *only on the first run*
                if not items_to_move_ids:
                    source_folder = archive_store.GetRootFolder()
                    items_to_move_ids = self.get_root_item_list(source_folder)

                    if not items_to_move_ids:
                        logging.info("No more MailItems found in the Online Archive root. Migration complete.")
                        break

                    logging.info(f"Found {len(items_to_move_ids)} MailItems to move initially.")

                    # Initial Confirmation for the whole process
                    if not messagebox.askyesno("Confirm Migration",
                        f"You are about to MOVE {len(items_to_move_ids)} item(s) from the Archive Root "
                        f"to the Inbox of '{archive_store.DisplayName}'.\n\n"
                        f"This process will use aggressive session resets (every {SESSION_RESET_THRESHOLD} moves).\n\n"
                        f"Do you want to proceed?"
                    ):
                        logging.warning("Migration cancelled by user.")
                        break

                # 4. Perform the move operation for a maximum of SESSION_RESET_THRESHOLD items
                moved_in_session = self.migrate_items(outlook, namespace, items_to_move_ids)

                # 5. Update the list of remaining IDs (remove items that were successfully moved)
                items_to_move_ids = items_to_move_ids[moved_in_session:]

                logging.info(f"Session finished. Successfully moved {moved_in_session} items in this run.")
                logging.info(f"Remaining items to move: {len(items_to_move_ids)}")

                # 6. Check for termination condition
                if not items_to_move_ids:
                    logging.info("All items processed. Migration is complete.")
                    break

                # 7. Check if a full session reset is required (always true if we hit a failure or the threshold)
                if moved_in_session < SESSION_RESET_THRESHOLD:
                    # If we stopped early due to an error, we reset immediately
                    logging.warning(f"Session failed early after {moved_in_session} moves. Forcing reset and cool-down.")
                else:
                    # If we moved exactly the threshold, we reset as planned
                    logging.info(f"Hit session threshold ({SESSION_RESET_THRESHOLD}). Forcing reset and cool-down.")

                # Force COM session cleanup and delay
                del outlook
                outlook = None # Ensure the object is released
                pythoncom.CoUninitialize()
                is_initialized = False # Will re-init on next loop iteration

                logging.info(f"Commencing long cool-down for {RECONNECT_DELAY_SECONDS} seconds...")
                time.sleep(RECONNECT_DELAY_SECONDS)
                logging.info("Cool-down finished. Attempting to restart session.")

            except Exception as e:
                logging.error(f"Critical error during session loop: {e}", exc_info=True)
                messagebox.showerror("Migration Error", f"A critical error occurred. Check logs for details: {e}")

                # Force cleanup and pause before trying again, in case the error was transient
                if outlook:
                    del outlook
                pythoncom.CoUninitialize()
                is_initialized = False

                logging.info(f"Pausing {RECONNECT_DELAY_SECONDS} seconds before attempting to restart due to critical error...")
                time.sleep(RECONNECT_DELAY_SECONDS)

        # Final Report Generation
        self.generate_report()
        logging.info("\nMigration process finished.")

        return self.migration_report['total_failed'] == 0

    def generate_report(self):
        """Generate a comprehensive migration report"""
        report_dir = "migration_reports"
        os.makedirs(report_dir, exist_ok=True)
        report_filename = os.path.join(report_dir, f'migration_report_{datetime.now().strftime("%Y%m%d_%H%M%S")}.json')

        self.migration_report['end_time'] = datetime.now().isoformat()
        if self.migration_report['start_time']:
            self.migration_report['duration_seconds'] = (
                datetime.fromisoformat(self.migration_report['end_time']) -
                datetime.fromisoformat(self.migration_report['start_time'])
            ).total_seconds()
        else:
            self.migration_report['duration_seconds'] = 0

        self.migration_report['success_rate'] = (
            self.migration_report['total_successful'] / self.migration_report['total_attempted'] * 100
        ) if self.migration_report['total_attempted'] > 0 else 0

        with open(report_filename, 'w') as f:
            json.dump(self.migration_report, f, indent=2, default=str)

        logging.info(f"Migration report saved to: {report_filename}")

        # Print summary to console
        print(f"\n=== MIGRATION SUMMARY ===")
        print(f"Start Time: {self.migration_report['start_time']}")
        print(f"End Time: {self.migration_report['end_time']}")
        print(f"Duration: {self.migration_report['duration_seconds']:.2f} seconds")
        print(f"Total Items Attempted: {self.migration_report['total_attempted']}")
        print(f"Successful Moves: {self.migration_report['total_successful']}")
        print(f"Failed Moves: {self.migration_report['total_failed']}")
        print(f"Success Rate: {self.migration_report['success_rate']:.2f}%")

        if self.migration_report['failed_items']:
            print(f"\n=== FAILED ITEMS ({len(self.migration_report['failed_items'])}) ===")
            print("Showing first 5 failures (check log file for all details):")
            for i, failed_item in enumerate(self.migration_report['failed_items'][:5], 1):
                print(f"{i}. Subject: {failed_item['subject'][:70]}...")
                print(f"   Error Type: {failed_item['type']}")
                print(f"   Error Details: {failed_item['error'][:70]}...")
                print("-" * 30)

# Usage
if __name__ == "__main__":
    print("\n" + "=" * 60)
    print("  Online Archive Root to Inbox Migrator (Aggressive Throttling)  ")
    print("=" * 60)
    print("This tool attempts to bypass server throttling by forcing a COM")
    print(f"session reset every {SESSION_RESET_THRESHOLD} moves with a {RECONNECT_DELAY_SECONDS} second cool-down.")
    print("Please ensure Outlook is running and the Online Archive is connected.")
    print("\n" + "=" * 60)

    migrator = EmailMigrator()
    success = migrator.run_migration()

    if success:
        print("\n✅ Migration completed successfully with no failed moves!")
    else:
        print("\n⚠️ Migration completed with failures. Check 'migration_logs' and 'migration_reports' folders for details.")